In [1]:
from tsai.all import *
from fastai.metrics import FBeta
from IPython.display import clear_output
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
my_setup()

os              : macOS-15.1-arm64-arm-64bit
python          : 3.11.7
tsai            : 0.4.0
fastai          : 2.7.19
fastcore        : 1.7.29
torch           : 2.5.1
device          : mps
cpu cores       : 14
threads per cpu : 1
RAM             : 24.0 GB
GPU memory      : N/A


In [2]:
path_results = "C:/Users/ryan30/OneDrive - Emory/Cognition-Assessment/code/tsfm-for-eda-main/moment/results"
batch_size = 256
num_workers = 4
num_eopchs = 5
lr = 0.1
archs = [(FCN, {}), (ResNet, {}), (ResCNN, {}), 
         (LSTM, {}), (TCN, {}), (InceptionTime, {}), (XceptionTime, {}), 
         (OmniScaleCNN, {}), (mWDN, {'levels': 4}), (TST, {})
         ]

In [ ]:
logging_train = pd.DataFrame(columns=['arch', 'hyperparams', 'total params', 'train loss', 'valid loss', 'accuracy', 'f1', 'time'])
logging_test = pd.DataFrame(columns=['arch', 'hyperparams', 'total params', 'fold', 'accuracy', 'precision', 'recall', 'f1'])
for i, (arch, k) in enumerate(archs):
    print(f"********************** {arch.__name__} **********************")
    for split in range(5):
        print(f"------------------- Fold {split} -------------------")
        dataset_path = f"C:/Users/ryan30/OneDrive - Emory/Cognition-Assessment/code/tsfm-for-eda-main/moment/data_preprocessing/WESAD_fine_tuning/split_fold_{split}"
        X_train = np.load(f"{dataset_path}/x_train.npy")
        y_train = np.load(f"{dataset_path}/y_train.npy")
        X_valid = np.load(f"{dataset_path}/x_val.npy")
        y_valid = np.load(f"{dataset_path}/y_val.npy")
        X_test = np.load(f"{dataset_path}/x_test.npy")
        y_test = np.load(f"{dataset_path}/y_test.npy")
    
        X, y, splits = combine_split_data([X_train, X_valid], [y_train, y_valid])
        tfms  = [None, [Categorize()]]
        dsets = TSDatasets(X, y, tfms=tfms, splits=splits, inplace=True)
        dls  = TSDataLoaders.from_dsets(dsets.train, dsets.valid, bs=[batch_size, batch_size*2], num_workers=num_workers)
        model = create_model(arch, dls=dls, **k)
        learn = Learner(dls, model,  metrics=[accuracy, FBeta(beta=1)])
        start = time.time()
        learn.fit_one_cycle(num_eopchs, lr)
        elapsed = time.time() - start
        vals = learn.recorder.values[-1]
        logging_train.loc[len(logging_train)] = [arch.__name__, k, count_parameters(model), vals[0], vals[1], vals[2], vals[3], elapsed]
        logging_train.to_csv(f"{path_results}/baseline_logging_train_0.1.csv", index=False)
        valid_dl = dls.valid
        X_test_3d = to3d(X_test)
        test_ds = valid_dl.dataset.add_test(X_test_3d, y_test)
        test_dl = valid_dl.new(test_ds)
        test_probas, test_targets, test_preds = learn.get_preds(dl=test_dl, with_decoded=True)
        acc = accuracy_score(test_targets, test_preds)
        precision, recall, f1, _ = precision_recall_fscore_support(test_targets, test_preds, average='binary')
        logging_test.loc[len(logging_test)] = [arch.__name__, k, count_parameters(model), split, acc, precision, recall, f1]
        logging_test.to_csv(f"{path_results}/baseline_logging_test_{lr}.csv", index=False)
        

********************** FCN **********************
------------------- Fold 0 -------------------


epoch,train_loss,valid_loss,accuracy,fbeta_score,time
0,0.433973,0.818213,0.666345,0.749819,00:14
1,0.415434,0.749719,0.641578,0.726163,00:10
2,0.406187,0.735480,0.558182,0.611803,00:10
3,0.360524,2.265501,0.515279,0.673402,00:10
4,0.317588,0.715029,0.723228,0.782830,00:10


------------------- Fold 1 -------------------


epoch,train_loss,valid_loss,accuracy,fbeta_score,time
0,0.472028,nan,0.823869,0.787070,00:10
1,0.501644,nan,0.525343,0.096482,00:10
2,0.488781,nan,0.522765,0.087095,00:10
3,0.462030,nan,0.524322,0.092775,00:10
4,0.444005,nan,0.521582,0.082756,00:10


------------------- Fold 2 -------------------


...

epoch,train_loss,valid_loss,accuracy,fbeta_score,time
0,0.480789,nan,0.667638,0.502182,00:09
1,0.445478,nan,0.698361,0.568122,00:18
2,0.358090,nan,0.608085,0.356810,00:25
3,0.399713,nan,0.594934,0.319767,00:18
4,0.376086,nan,0.584473,0.289057,00:08


------------------- Fold 2 -------------------


epoch,train_loss,valid_loss,accuracy,fbeta_score,time
0,nan,nan,0.500000,0.000000,00:08
1,nan,nan,0.500000,0.000000,00:08
2,nan,nan,0.500000,0.000000,00:09
3,nan,nan,0.500000,0.000000,00:23
4,nan,nan,0.500000,0.000000,00:25


------------------- Fold 3 -------------------


epoch,train_loss,valid_loss,accuracy,fbeta_score,time
0,nan,nan,0.500000,0.000000,00:08
1,nan,nan,0.500000,0.000000,00:08
2,nan,nan,0.500000,0.000000,00:09
3,nan,nan,0.500000,0.000000,00:21
